In [1]:
import os
import numpy as np
from scipy.io import loadmat,savemat
import nibabel as nib
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import torch
import scipy.io as sio

bs_size = 1000

# v2: seperate training and testing and save as text
# v3: include all 10 datasets and concatenate!

In [2]:
dirs = ['A','B','C','D','E','F','G','I','J','K']

X = np.empty((3,3,3,15,0))
Y = np.empty((1,1,1,15,0))

for i in range(len(dirs)):
    data_path = "/Users/kurtschilling/Data/harmonization/s%s_A2C_Patches.mat" % (dirs[i])
    print(data_path)
    mat_contents = sio.loadmat(data_path)
    # mat_contents
    Xv = mat_contents['input3000']
    print(X.shape)
    Yv = mat_contents['output3000']
    print(Yv.shape)
    print(Xv.dtype)
    print(Yv.dtype)
    
    # remove NAN, INF
    from numpy import asarray as ar

    arr1 = np.squeeze(np.isnan(Yv).any(axis=3))
    arr2 = np.squeeze(np.isinf(Yv).any(axis=3))
    arr3 = ar(arr1) | ar(arr2)
    print(arr3.shape)

    #X = np.delete(X,arr3,4)
    #Y = np.delete(Y,arr3,4)
    Xv = Xv[:,:,:,:,~arr3]
    Yv = Yv[:,:,:,:,~arr3]
    print(Xv.shape)
    print(Yv.shape)

    # remove >10
    arr6 = np.squeeze(np.greater(np.abs(Yv),10).any(axis=3))
    print(arr6.shape)
    #X = np.delete(X,arr6,4)
    #Y = np.delete(Y,arr6,4)
    Xv = Xv[:,:,:,:,~arr6]
    Yv = Yv[:,:,:,:,~arr6]
    print(Xv.shape)
    print(Yv.shape)

    X = np.append(X, Xv, axis=4)
    Y = np.append(Y, Yv, axis=4)

/Users/kurtschilling/Data/harmonization/sA_A2C_Patches.mat
(3, 3, 3, 15, 0)
(1, 1, 1, 15, 303789)
float64
float64
(303789,)
(3, 3, 3, 15, 303786)
(1, 1, 1, 15, 303786)
(303786,)
(3, 3, 3, 15, 303756)
(1, 1, 1, 15, 303756)
/Users/kurtschilling/Data/harmonization/sB_A2C_Patches.mat
(3, 3, 3, 15, 303756)
(1, 1, 1, 15, 336090)
float64
float64
(336090,)
(3, 3, 3, 15, 336087)
(1, 1, 1, 15, 336087)
(336087,)
(3, 3, 3, 15, 336054)
(1, 1, 1, 15, 336054)
/Users/kurtschilling/Data/harmonization/sC_A2C_Patches.mat
(3, 3, 3, 15, 639810)
(1, 1, 1, 15, 318295)
float64
float64
(318295,)
(3, 3, 3, 15, 318279)
(1, 1, 1, 15, 318279)
(318279,)
(3, 3, 3, 15, 318234)
(1, 1, 1, 15, 318234)
/Users/kurtschilling/Data/harmonization/sD_A2C_Patches.mat
(3, 3, 3, 15, 958044)
(1, 1, 1, 15, 297370)
float64
float64
(297370,)
(3, 3, 3, 15, 297364)
(1, 1, 1, 15, 297364)
(297364,)
(3, 3, 3, 15, 297337)
(1, 1, 1, 15, 297337)
/Users/kurtschilling/Data/harmonization/sE_A2C_Patches.mat
(3, 3, 3, 15, 1255381)
(1, 1, 1, 15, 3

In [3]:
print(X.shape)
print(Y.shape)
print(np.amax(X.shape))
print(np.amax(Xv.shape))

(3, 3, 3, 15, 3208879)
(1, 1, 1, 15, 3208879)
3208879
321709


In [4]:
from torch.utils.data import Dataset
from torchvision import transforms

class SHDataSet(Dataset):
    def __init__(self,X,Y):
        
        self.X = X
        self.Y = Y
        self.to_tensor = transforms.ToTensor()
        
    def __len__(self):
        return np.amax(X.shape)
    
    def __getitem__(self, i):
        
        #vec_a = np.reshape(self.X[i,:],(15, 1, 1))
        #vec_b = np.reshape(self.Y[i,:],(15, 1, 1))
        vec_a = self.X[:,:,:,:,i]
        vec_b = self.Y[:,:,:,:,i]
        #vec_a = np.reshape(self.X[i,:],(1, 15, 1))
        #vec_b = np.reshape(self.Y[i,:],(1, 15, 1))
        vec_a = np.transpose(vec_a, (3, 0, 1, 2))
        vec_b = np.transpose(vec_b, (3, 0, 1, 2))
        #a = self.to_tensor(vec_a)
        #b = self.to_tensor(vec_b)
        a = torch.Tensor(vec_a)
        b = torch.Tensor(vec_b.squeeze())
    
        return a,b
    


In [5]:
shset = SHDataSet(X,Y)
print(len(shset))

3208879


In [6]:
print(X.shape)
print(Y.shape)
print(X.dtype)
print(Y.dtype)

(3, 3, 3, 15, 3208879)
(1, 1, 1, 15, 3208879)
float64
float64


In [7]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #self.fc1 = nn.Linear(3 * 3 * 3 * 15, 600)
        self.cn1 = nn.Conv3d(15,128,kernel_size=(3,3,3),stride=1,padding=(1,1,1))
        #print(self.cn1.shape)
        #self.fc2 = nn.Linear(600,300)
        self.bn = nn.BatchNorm3d(128)
        self.fc2 = nn.Linear(128*3*3*3, 300)
        self.fc3 = nn.Linear(300,60)
        self.fc4 = nn.Linear(60,200)
        self.fc5 = nn.Linear(200,15)
        
    def forward(self, x):
        x = F.relu(self.cn1(x))
        print(x.shape)
        dimensions = x.shape
        x = self.bn(x)
        x = x.view(dimensions[0], -1)
        #x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x
        
net = Net()
print(net)  

optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
criterion = nn.MSELoss()

Net(
  (cn1): Conv3d(15, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (bn): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=3456, out_features=300, bias=True)
  (fc3): Linear(in_features=300, out_features=60, bias=True)
  (fc4): Linear(in_features=60, out_features=200, bias=True)
  (fc5): Linear(in_features=200, out_features=15, bias=True)
)


In [8]:
from __future__ import print_function, division
import torch

def train(model, device, loader, optimizer):
    model.train()

    correct = 0
    total_loss = 0

    for batch_idx, (data, target) in enumerate(loader):
        data, target = Variable(data).float(), Variable(target).float()
        
        optimizer.zero_grad()
        output = model(data)
        output = output.to(device)

        loss = criterion(output,target)
        total_loss += loss.item()
      
        loss.backward()
        optimizer.step()
        
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx*len(data),len(shset.X),100.*batch_idx/len(shset.X),loss.data[0]))

    avg_loss = total_loss / batch_idx
    print('\tTraining set: Average loss: {:.4f}'.format(avg_loss))
   
    return avg_loss

In [9]:
def test(model, device, loader):
    model.eval()

    correct = 0
    total_loss = 0

    for batch_idx, (data, target) in enumerate(loader):
        data, target = Variable(data).float(), Variable(target).float()

        output = model(data)
        output = output.to(device)

        loss = criterion(output, target)
        total_loss += loss.item()

    avg_loss = total_loss / batch_idx
    print('\tTesting set: Average loss: {:.4f}'.format(avg_loss))

    return avg_loss



In [10]:
# batch_size = 16
validation_split = .2
shuffle_dataset = True
random_seed= 42


# Creating data indices for training and validation splits:
dataset_size = len(shset)
print(dataset_size)

indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

from torch.utils.data.sampler import SubsetRandomSampler

use_cuda = torch.cuda.is_available()
torch.manual_seed(1)
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = DataLoader(shset, batch_size=bs_size, sampler=train_sampler)
validation_loader = DataLoader(shset, batch_size=bs_size, sampler=valid_sampler)



3208879


In [11]:
import random

train_loss_file = 'BLAHFCN_PATCH_A2C_SH3000_train_loss_split.txt'
f = open(train_loss_file, 'w')
f.close()
validate_loss_file = 'BLAHFCN_PATCH_A2C_SH3000_validate_loss_split.txt'
f = open(validate_loss_file, 'w')
f.close()

model_file = 'BLAHFCN_PATCH_A2C_SH3000_saved_model_split'

In [ ]:
for epoch in range(1, 51):
        print('\nEpoch %d: ' % epoch)
        loss = train(net, device, train_loader, optimizer)

        with open(train_loss_file, "a") as file:
            file.write(str(loss))
            file.write('\n')

        loss = test(net, device, validation_loader)

        with open(validate_loss_file, "a") as file:
            file.write(str(loss))
            file.write('\n')

        if epoch % 5 == 0:
            with open(model_file, 'wb') as f:
                torch.save(net.state_dict(), f)


Epoch 1: 
torch.Size([1000, 128, 3, 3, 3])


/anaconda3/envs/python36/lib/python3.6/site-packages/ipykernel_launcher.py:23: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Train Epoch: 1 [0/3 (0%)]	Loss: 0.033800
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [1000/3 (33%)]	Loss: 0.033665
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [2000/3 (67%)]	Loss: 0.032798
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [3000/3 (100%)]	Loss: 0.032107
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [4000/3 (133%)]	Loss: 0.031037
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [5000/3 (167%)]	Loss: 0.032534
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [6000/3 (200%)]	Loss: 0.032152
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [7000/3 (233%)]	Loss: 0.031863
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [8000/3 (267%)]	Loss: 0.032942
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [9000/3 (300%)]	Loss: 0.036993
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [10000/3 (333%)]	Loss: 0.031694
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [11000/3 (367%)]	Loss: 0.032961
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [12000/3 (400%)]	Loss: 0.032504
torch.Size

Train Epoch: 1 [102000/3 (3400%)]	Loss: 0.014009
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [103000/3 (3433%)]	Loss: 0.012232
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [104000/3 (3467%)]	Loss: 0.012106
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [105000/3 (3500%)]	Loss: 0.011484
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [106000/3 (3533%)]	Loss: 0.012520
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [107000/3 (3567%)]	Loss: 0.011053
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [108000/3 (3600%)]	Loss: 0.011334
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [109000/3 (3633%)]	Loss: 0.012138
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [110000/3 (3667%)]	Loss: 0.011464
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [111000/3 (3700%)]	Loss: 0.010952
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [112000/3 (3733%)]	Loss: 0.010608
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [113000/3 (3767%)]	Loss: 0.010365
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [

Train Epoch: 1 [202000/3 (6733%)]	Loss: 0.004736
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [203000/3 (6767%)]	Loss: 0.004662
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [204000/3 (6800%)]	Loss: 0.004659
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [205000/3 (6833%)]	Loss: 0.004625
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [206000/3 (6867%)]	Loss: 0.005104
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [207000/3 (6900%)]	Loss: 0.004273
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [208000/3 (6933%)]	Loss: 0.004559
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [209000/3 (6967%)]	Loss: 0.004227
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [210000/3 (7000%)]	Loss: 0.004362
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [211000/3 (7033%)]	Loss: 0.004098
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [212000/3 (7067%)]	Loss: 0.004837
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [213000/3 (7100%)]	Loss: 0.011227
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [

Train Epoch: 1 [302000/3 (10067%)]	Loss: 0.003706
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [303000/3 (10100%)]	Loss: 0.003581
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [304000/3 (10133%)]	Loss: 0.006078
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [305000/3 (10167%)]	Loss: 0.004231
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [306000/3 (10200%)]	Loss: 0.003843
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [307000/3 (10233%)]	Loss: 0.004470
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [308000/3 (10267%)]	Loss: 0.003606
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [309000/3 (10300%)]	Loss: 0.003887
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [310000/3 (10333%)]	Loss: 0.004474
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [311000/3 (10367%)]	Loss: 0.003633
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [312000/3 (10400%)]	Loss: 0.004524
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [313000/3 (10433%)]	Loss: 0.005959
torch.Size([1000, 128, 3, 3, 3])
Trai

Train Epoch: 1 [401000/3 (13367%)]	Loss: 0.003847
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [402000/3 (13400%)]	Loss: 0.003383
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [403000/3 (13433%)]	Loss: 0.003447
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [404000/3 (13467%)]	Loss: 0.003709
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [405000/3 (13500%)]	Loss: 0.003255
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [406000/3 (13533%)]	Loss: 0.003224
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [407000/3 (13567%)]	Loss: 0.003755
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [408000/3 (13600%)]	Loss: 0.004015
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [409000/3 (13633%)]	Loss: 0.003511
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [410000/3 (13667%)]	Loss: 0.005148
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [411000/3 (13700%)]	Loss: 0.003527
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [412000/3 (13733%)]	Loss: 0.003297
torch.Size([1000, 128, 3, 3, 3])
Trai

Train Epoch: 1 [500000/3 (16667%)]	Loss: 0.007729
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [501000/3 (16700%)]	Loss: 0.005633
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [502000/3 (16733%)]	Loss: 0.003131
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [503000/3 (16767%)]	Loss: 0.003579
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [504000/3 (16800%)]	Loss: 0.003829
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [505000/3 (16833%)]	Loss: 0.003379
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [506000/3 (16867%)]	Loss: 0.003746
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [507000/3 (16900%)]	Loss: 0.003109
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [508000/3 (16933%)]	Loss: 0.003410
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [509000/3 (16967%)]	Loss: 0.003314
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [510000/3 (17000%)]	Loss: 0.004823
torch.Size([1000, 128, 3, 3, 3])
Train Epoch: 1 [511000/3 (17033%)]	Loss: 0.003561
torch.Size([1000, 128, 3, 3, 3])
Trai